In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [ ]:
%matplotlib inline
pd.options.display.float_format = '{:.5f}'.format

## Load file

In [ ]:
file = "../data/0_raw/data.csv" 
df_raw = pd.read_csv(file
                   , sep=','
                   , header=0
                   , parse_dates=['data_col']
                   , infer_datetime_format=True
                   , low_memory=False)
df_raw.head()

In [ ]:
df_raw.info(show_counts=True)

## Analysis of dataset

In [ ]:
# df would be a copy of raw data that would be processed
df = df_raw.copy()

### General look

In [ ]:
df.describe(include='all', datetime_is_numeric=True)

### Target
Code target and subset to relevant rows

In [ ]:
df['target_old_name'].value_counts()

In [ ]:
# create column target where 1 is default, 0 is not, -1 is out of analysis
target_map = {"cat 0": 0, "cat 1":1, "cat irrelevant": -1}

In [ ]:
df['target'] = df['target_old_name'].map(target_map)
df.drop(columns=['target_old_name'], inplace=True)

In [ ]:
df['target'].value_counts()

In [ ]:
df = df.query('target >= 0').reset_index(drop=True)

In [ ]:
sns.countplot(x=df['target'], palette='viridis')

### Not relevant features
There are some feautres which are not relevant

In [ ]:
cols_to_drop=['co1', 'col2']
df.drop(columns=cols_to_drop, inplace=True)

### Duplication in data
There is no ID column so using all columns to check duplicates

In [ ]:
# check if there are some rows that have the same values for all columns except target
df[df.drop(columns=['target']).duplicated(keep=False)]

Conclusions:
1. 
2. 
3. 


### Missing values

In [ ]:
df.isna().sum()

In [ ]:
df.isna().sum() / len(df)

#### missing col3

In [ ]:
# col 3 check
df[pd.isna(df['col3'])]

#### missings values solution
- missings for col3 - delete rows 

In [ ]:
df.dropna(axis=0, subset=['col3'], inplace=True)

In [ ]:
df.isna().sum()

## Distribution of data

In [ ]:
col_float = list(df.select_dtypes(include=['float64']).columns)
col_obj = list(df.select_dtypes(include=['object']).columns)

In [ ]:
for col in col_float:
    print(f'Distribution: {col}')
    sns.boxplot(y=df[col], x=df['target'])
    plt.show()

In [ ]:
for col in col_obj:
    print(f'Distribution: {col}')
    plt.figure(figsize=(16,4))
    sns.countplot(x=df[col],hue=df['target'], order=sorted(df[col].unique()))
    plt.show()

In [ ]:
print(f'Distribution: date_col')
plt.figure(figsize=(20,4))
ax = sns.countplot(x=df['date_col'].dt.year,hue=df['target'], order=sorted(df['date_col'].dt.year.unique()))
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
plt.show()

Conclusions:
- there are outliers to detect
- other conclusions

## Encode

In [ ]:
col_map = {'val1': 0, ' val2': 1, 'vl3': 1}
df['col_new'] = df['col_old'].map(col_map)

In [ ]:
df = pd.get_dummies(df, columns=['col5'], drop_first=True)

## Correlation

#### Between numerical

In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(df.corr(),annot=True)
plt.show()

In [ ]:
# check correlated pairs
plt.figure(figsize=(15, 5))
sns.histplot(x=df['one'], hue=df['two'], bins=20)

Conclusion:
- remove two as is correlated with one

In [ ]:
df.drop(columns=['two'], inplace=True)

## Outliers

### log transformation

In [ ]:
# check if log transformation helps with outliers
for col in col_float:
    print(f'Distribution: log of {col}')
    sns.boxplot(y=np.log(df[col]+0.0001), x=df['target'])
    plt.show()

In [ ]:
# transform to log
for col in col_float:
    df[col] = df[col].apply(lambda x: np.log(x+0.00001))

In [ ]:
df.describe(include='all')

## Sum up analysis

1. target variable to code as binary and drop not relevant values
2. no duplicates to drop
3. drop features:
    - not relevant: cols_to_drop=['col2', 'col'] 
    - correlated: one_a, one_b
4. missings:
    - missings for col - delete row
5. encode cat values
        - col_map = {'val1': 0, ' val2': 1, 'vl3': 1}
        - col5 - dummies
6. transformations:
    - date_col - only year 
    - log numerical variables 
    - remove outliers